In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
data=pd.read_csv("all.csv",index_col=None )

In [3]:
data.describe()

,author,content,poem name,age,type
count,573,573,571,573,573
unique,67,506,508,2,3
top,WILLIAM SHAKESPEARE,"Originally published in Poetry, March 1914.",Canto IV,Renaissance,Love
freq,71,4,3,315,326


In [4]:
data.head()

,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore


#Data cleaning (since the data is from web crawler, it has some faults)

In [5]:
data=data[data.content.str.contains("Copyright")==False]

In [6]:
data

,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore
5,RICHARD BARNFIELD,"Cherry-lipt Adonis in his snowie shape,\r\n ...",Sonnet 17,Renaissance,Mythology & Folklore
6,SIR WALTER RALEGH,Praisd be Dianas fair and harmless light;\r\nP...,Praisd be Dianas Fair and Harmless Light,Renaissance,Mythology & Folklore
7,QUEEN ELIZABETH I,"When I was fair and young, then favor graced m...",When I Was Fair and Young,Renaissance,Mythology & Folklore
8,JOHN DONNE,"When by thy scorn, O murd'ress, I am dead\r\n ...",The Apparition,Renaissance,Mythology & Folklore
9,JOHN SKELTON,"Pla ce bo,\r\nWho is there, who?\r\nDi le xi,\...",The Book of Phillip Sparrow,Renaissance,Mythology & Folklore


In [7]:
# check whether there is any null data 
data[data.content.isnull()==True].index.tolist()

[]

In [8]:
# This is the fault data checked by my eye in csv file, so remove it  
data=data[data.content.str.contains("from Dana, 1904")==False]

In [9]:
import nltk
nltk.download('cmudict')
entries = nltk.corpus.cmudict.entries()
print(entries)
len(entries)

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


[('a', ['AH0']), ('a.', ['EY1']), ('a', ['EY1']), ...]


133737

In [10]:
def rhyme(inp, level):
    entries = nltk.corpus.cmudict.entries()
    syllables = [(word, syl) for word, syl in entries if word == inp]
    rhymes = []
    for (word, syllable) in syllables:
        rhymes += [word for word, pron in entries if pron[-level:] == syllable[-level:]]
    return set(rhymes)

In [11]:
def doTheyRhyme ( word1, word2 ):
    # first, we don't want to report 'glue' and 'unglue' as rhyming words
    # those kind of rhymes are LAME
    if word1.find ( word2 ) == len(word1) - len ( word2 ):
#         print(word1.find(word2))
        return False
    if word2.find ( word1 ) == len ( word2 ) - len ( word1 ): 
#         print(word2.find(word1))
        return False

    return word1 in rhyme ( word2, 1 )

In [12]:
doTheyRhyme('glue', 'unglue')

False

# Rhyme, similes, repetitions, alliterations of the first poem 

In [13]:
# First poem in our csv file
data.content[0]

'Let the bird of loudest lay\r\nOn the sole Arabian tree\r\nHerald sad and trumpet be,\r\nTo whose sound chaste wings obey.\r\n\r\nBut thou shrieking harbinger,\r\nFoul precurrer of the fiend,\r\nAugur of the fever\'s end,\r\nTo this troop come thou not near.\r\n\r\nFrom this session interdict\r\nEvery fowl of tyrant wing,\r\nSave the eagle, feather\'d king;\r\nKeep the obsequy so strict.\r\n\r\nLet the priest in surplice white,\r\nThat defunctive music can,\r\nBe the death-divining swan,\r\nLest the requiem lack his right.\r\n\r\nAnd thou treble-dated crow,\r\nThat thy sable gender mak\'st\r\nWith the breath thou giv\'st and tak\'st,\r\n\'Mongst our mourners shalt thou go.\r\n\r\nHere the anthem doth commence:\r\nLove and constancy is dead;\r\nPhoenix and the Turtle fled\r\nIn a mutual flame from hence.\r\n\r\nSo they lov\'d, as love in twain\r\nHad the essence but in one;\r\nTwo distincts, division none:\r\nNumber there in love was slain.\r\n\r\nHearts remote, yet not asunder;\r\nDis

make it more readable

In [14]:
lin=data.content[0].splitlines()
for li in lin:
    print(li)

Let the bird of loudest lay
On the sole Arabian tree
Herald sad and trumpet be,
To whose sound chaste wings obey.

But thou shrieking harbinger,
Foul precurrer of the fiend,
Augur of the fever's end,
To this troop come thou not near.

From this session interdict
Every fowl of tyrant wing,
Save the eagle, feather'd king;
Keep the obsequy so strict.

Let the priest in surplice white,
That defunctive music can,
Be the death-divining swan,
Lest the requiem lack his right.

And thou treble-dated crow,
That thy sable gender mak'st
With the breath thou giv'st and tak'st,
'Mongst our mourners shalt thou go.

Here the anthem doth commence:
Love and constancy is dead;
Phoenix and the Turtle fled
In a mutual flame from hence.

So they lov'd, as love in twain
Had the essence but in one;
Two distincts, division none:
Number there in love was slain.

Hearts remote, yet not asunder;
Distance and no space was seen
'Twixt this Turtle and his queen:
But in them it were a wonder.

So between them love di

In [15]:
lin=(data.content[0]).replace(","," ").replace("."," ").replace(";"," ").replace(":"," ").replace("!"," ")
line=lin.splitlines()
i=0
Set=[]
Sentence_set=[]
result=""
print ("---last word of the sentence, see the below for the result ---")
for li in line:
    print("li", li)
    Sentence_set.append(li)
    sp=li.split()
    print("sp", sp)
    print("sp-1", sp[-1:])
    if ("".join(sp[-1:]) is None or  len("".join(sp[-1:])) == 0 or "".join(sp[-1:])==" "):
        print("nhi jaaega")
        continue
    print (str(i)+" "+"".join(sp[-1:]))
    Set.append("".join(sp[-1:]))
    print(Set)
    if(len(Set)%4==0):
        print("iske andar")
        if(doTheyRhyme(Set[len(Set)-4], Set[len(Set)-3] )== True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==True and doTheyRhyme(Set[len(Set)-2], Set[len(Set)-1] )==True):
            result= ("--AAAA--");
        elif (  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-3] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==False and doTheyRhyme(Set[len(Set)-2], Set[len(Set)-1] )==True):
            result =("--AABB--");
        elif (  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-2] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-1] )==False and doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==False):
            result =("--ABAB--");
        elif (  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-1] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-1] )==False and doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==True):
#             print("yaha ghusa lay tree be obey abba")
            result =("--ABBA--");
    i=i+1
    print(i)
if(result==""):    
    result= ("--No Rhyme--");
print(result)
    



---last word of the sentence, see the below for the result ---
li Let the bird of loudest lay
sp ['Let', 'the', 'bird', 'of', 'loudest', 'lay']
sp-1 ['lay']
0 lay
['lay']
1
li On the sole Arabian tree
sp ['On', 'the', 'sole', 'Arabian', 'tree']
sp-1 ['tree']
1 tree
['lay', 'tree']
2
li Herald sad and trumpet be 
sp ['Herald', 'sad', 'and', 'trumpet', 'be']
sp-1 ['be']
2 be
['lay', 'tree', 'be']
3
li To whose sound chaste wings obey 
sp ['To', 'whose', 'sound', 'chaste', 'wings', 'obey']
sp-1 ['obey']
3 obey
['lay', 'tree', 'be', 'obey']
iske andar
4
li 
sp []
sp-1 []
nhi jaaega
li But thou shrieking harbinger 
sp ['But', 'thou', 'shrieking', 'harbinger']
sp-1 ['harbinger']
4 harbinger
['lay', 'tree', 'be', 'obey', 'harbinger']
5
li Foul precurrer of the fiend 
sp ['Foul', 'precurrer', 'of', 'the', 'fiend']
sp-1 ['fiend']
5 fiend
['lay', 'tree', 'be', 'obey', 'harbinger', 'fiend']
6
li Augur of the fever's end 
sp ['Augur', 'of', 'the', "fever's", 'end']
sp-1 ['end']
6 end
['lay', 'tree

32
li 
sp []
sp-1 []
nhi jaaega
li So between them love did shine
sp ['So', 'between', 'them', 'love', 'did', 'shine']
sp-1 ['shine']
32 shine
['lay', 'tree', 'be', 'obey', 'harbinger', 'fiend', 'end', 'near', 'interdict', 'wing', 'king', 'strict', 'white', 'can', 'swan', 'right', 'crow', "mak'st", "tak'st", 'go', 'commence', 'dead', 'fled', 'hence', 'twain', 'one', 'none', 'slain', 'asunder', 'seen', 'queen', 'wonder', 'shine']
33
li That the Turtle saw his right
sp ['That', 'the', 'Turtle', 'saw', 'his', 'right']
sp-1 ['right']
33 right
['lay', 'tree', 'be', 'obey', 'harbinger', 'fiend', 'end', 'near', 'interdict', 'wing', 'king', 'strict', 'white', 'can', 'swan', 'right', 'crow', "mak'st", "tak'st", 'go', 'commence', 'dead', 'fled', 'hence', 'twain', 'one', 'none', 'slain', 'asunder', 'seen', 'queen', 'wonder', 'shine', 'right']
34
li Flaming in the Phoenix' sight 
sp ['Flaming', 'in', 'the', "Phoenix'", 'sight']
sp-1 ['sight']
34 sight
['lay', 'tree', 'be', 'obey', 'harbinger', 'fi

52
li 
sp []
sp-1 []
nhi jaaega
li                  threnos
sp ['threnos']
sp-1 ['threnos']
52 threnos
['lay', 'tree', 'be', 'obey', 'harbinger', 'fiend', 'end', 'near', 'interdict', 'wing', 'king', 'strict', 'white', 'can', 'swan', 'right', 'crow', "mak'st", "tak'st", 'go', 'commence', 'dead', 'fled', 'hence', 'twain', 'one', 'none', 'slain', 'asunder', 'seen', 'queen', 'wonder', 'shine', 'right', 'sight', 'mine', 'appalled', 'same', 'name', 'called', 'confounded', 'together', 'neither', 'compounded', 'twain', 'one', 'none', '"', 'threne', 'Dove', 'love', 'scene', 'threnos']
53
li 
sp []
sp-1 []
nhi jaaega
li Beauty  truth  and rarity 
sp ['Beauty', 'truth', 'and', 'rarity']
sp-1 ['rarity']
53 rarity
['lay', 'tree', 'be', 'obey', 'harbinger', 'fiend', 'end', 'near', 'interdict', 'wing', 'king', 'strict', 'white', 'can', 'swan', 'right', 'crow', "mak'st", "tak'st", 'go', 'commence', 'dead', 'fled', 'hence', 'twain', 'one', 'none', 'slain', 'asunder', 'seen', 'queen', 'wonder', 'shine',

68
--ABBA--


In [16]:
print ("--sentences for similes  in first poem--")
for sen in Sentence_set:
    sen_break = sen.split();
    if (( "like"  in sen_break) or ("as"  in sen_break)):
        print (sen)


--sentences for similes  in first poem--
So they lov'd  as love in twain


In [17]:
print ("--sentences for alliterations  in first poem--")
for sen in Sentence_set:
    sen_break = sen.split();
    if (sen_break is None or  len(sen_break) == 0 or sen_break==" "):
        continue
    counter=1
    threeorfour=1
    last_character=''
    for ele_in in sen_break:
        if (ele_in[0]==last_character):
            threeorfour+=1
            if(counter==3):
                print (" ".join(sen_break) )
        if(counter==1):
            last_character=ele_in[0]
        counter+=1

--sentences for alliterations  in first poem--
That the Turtle saw his right


In [ ]:
print ("--sentences for repetitions   in first poem--")
for sen in Sentence_set:
    sen_break = sen.split();
    if (sen_break is None or  len(sen_break) == 0 or sen_break==" "):
        continue
    for ele_in in sen_break:
        sen_break.remove(ele_in)
        if (ele_in in sen_break):
                print (sen +"      -word of Repetitions:"+ ele_in)
        

#Convert data into feature 

 1. Turn all the sentence into lower case 
 2. Remove punctuation and Line break
 3. Remove the word of the list in doc
 4.  Find the word map using tfidfvectorizer library



In [ ]:
data.content.str.lower()

In [ ]:
data.content=data.content.str.replace('\n', " ")
data.content=data.content.str.replace("\t", " ")
data.content=data.content.str.replace("\r", " ")
data.content=data.content.str.replace(","," ").replace("."," ")


In [ ]:
# remove list in the document
remove_list=["A",
"An",
"The",
"Aboard",
"About",
"Above",
"Absent",
"Across",
"After",
"Against",
"Along",
"Alongside",
"Amid",
"Among",
"Amongst",
"Anti",
"Around",
"As",
"At",
"Before",
"Behind",
"Below",
"Beneath",
"Beside",
"Besides",
"Between",
"Beyond",
"But",
"By",
"Circa",
"Concerning",
"Considering",
"Despite",
"Down",
"During",
"Except",
"Excepting",
"Excluding",
"Failing",
"Following",
"For",
"From",
"Given",
"In",
"Inside",
"Into",
"Like",
"Minus",
"Near",
"Of",
"Off",
"On",
"Onto",
"Opposite",
"Outside",
"Over",
"Past",
"Per",
"Plus",
"Regarding",
"Round",
"Save",
"Since",
"Than",
"Through",
"To",
"Toward",
"Towards",
"Under",
"Underneath",
"Unlike",
"Until",
"Up",
"Upon",
"Versus",
"Via",
"With",
"Within",
"Without"]

In [ ]:
# replace those words with space
for  value in remove_list:
    data.content=data.content.str.replace(value," ")
data.content

In [ ]:
import re
# regular expression, using stemming: try to replace tail of words like ies to y 

In [ ]:
   data.content = data.content.str.replace("ing( |$)", " ")
data.content = data.content.str.replace("[^a-zA-Z]", " ")
data.content = data.content.str.replace("ies( |$)", "y ")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, analyzer= 'word')

In [ ]:
data.head()

In [ ]:
data[["content","author","poem name"]]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# if there is any empty data, drop it
data.dropna(inplace=True)

#Split the data: 80 % for training data and  20 % for testing data

In [ ]:

data_content_train,data_content_test, data_train_label,data_test_label =train_test_split(data[["content","author","poem name"]],data.type,test_size = 0.2, random_state = 1)


In [ ]:
data_content_train

In [33]:
train_ = vectorizer.fit_transform(data_content_train.content.as_matrix())
feature_names =vectorizer.get_feature_names()
feature_names
test_ = vectorizer.transform(data_content_test.content.as_matrix())

C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [34]:
# check if there is any empty poem name in the file
removelist=data_content_train["poem name"].index[data_content_train["poem name"].isnull()==True].tolist()
removelist

[]

#Our targets contain :
- type (Love, Nature, Mythology & Folklore)

In [35]:
from sklearn import preprocessing
label_au = preprocessing.LabelEncoder()
label_author=label_au.fit_transform(data_content_train.author.as_matrix())
label_authorT=label_au.fit_transform(data_content_test.author.as_matrix())

label_poe_name =TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')  
label_poena=label_poe_name.fit_transform(data_content_train["poem name"].as_matrix())
label_poenaT  =label_poe_name.fit_transform(data_content_test["poem name"].as_matrix())



C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [36]:
label_author=np.reshape(label_author, (label_author.shape[0], 1))
label_authorT=np.reshape(label_authorT, (label_authorT.shape[0], 1))

In [37]:
from numpy import array

In [38]:
# We try to catch more feature, but it did not make big difference of result
from sklearn.feature_selection import SelectKBest ,chi2
#y = np.array(data_content_train)
ch2 = SelectKBest(chi2, k=2000)
#X_train=ch2.fit_transform(train_, data_train_label.tolist() )
#X_test = ch2.transform(test_)
X_train=train_;
X_test=test_;

In [39]:
# It did not make big difference of result if we use dense matrix 
#import scipy.sparse as sp
#if(sp.issparse(X_train)==True):
#   X_train = X_train.todense()
#   X_test = X_test.todense()
    

### Classifier 1: XGBoost

In [40]:
import xgboost as xgb

In [41]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    "eval_metric": 'logloss',
    'silent': 1
}

In [42]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
a=le.fit_transform(data_train_label.as_matrix())

C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
dtrain = xgb.DMatrix(X_train, a )
dtest = xgb.DMatrix(X_test)

In [44]:
num_boost_rounds = 422
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
result =model.predict(dtest)

[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=6
[13:35:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_

[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_

[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=6
[13:35:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

[13:35:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[13:35:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_

[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[13:35:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_

In [45]:
presult=pd.DataFrame(result)

In [46]:
presult[(presult.values >= 0.5) & (presult.values < 1.5) ]= 1;
presult[(presult.values >= 1.5) & (presult.values < 2.5) ]=2;
presult[(presult.values >= -0.5) & (presult.values < 0.5) ]=0;

In [47]:
presult=presult.astype(int)

In [48]:
result_back=le.inverse_transform(presult.values)

C:\Users\LENOVO\Anaconda3\envs\tf-keras\lib\site-packages\sklearn\preprocessing\label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
# accuracy for target type 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(data_test_label, result_back)
accuracy

0.47

#Predicting poem type 

In [50]:
pd.DataFrame({  'poem name': data_content_test["poem name"],
                'correct_data' : data_test_label,
                'predict result' : result_back.ravel()
                    })

,poem name,correct_data,predict result
304,There Is A Garden In Her Face,Love,Love
351,The Wife,Mythology & Folklore,Nature
47,"Astrophil and Stella 31: With how sad steps, O...",Nature,Love
67,Song: Under the greenwood tree,Nature,Mythology & Folklore
551,Gloire de Dijon,Love,Nature
557,A Negro Love Song,Love,Love
310,What Needeth these Threat'ning Words,Love,Love
31,"Fie, Pleasure, Fie!",Nature,Mythology & Folklore
249,Elegy V: His Picture,Love,Nature
90,from Pamphilia to Amphilanthus: 7,Love,Love


2nd classifier

In [ ]:
# from nltk.corpus import stopwords
import re
# import string
import matplotlib.pyplot as plt
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import normalize
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
# from sklearn import svm
from subprocess import check_output
# from wordcloud import WordCloud, STOPWORDS


from sklearn.linear_model import LogisticRegression
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
classifier = OneVsRestClassifier(LogisticRegression(penalty='l1'), n_jobs=-1)
classifier.fit(data_content_train, data_train_label)
predictions = classifier.predict(data_content_test)

print("accuracy :",metrics.accuracy_score(data_test_label,predictions))
print("macro f1 score :",metrics.f1_score(data_test_label, predictions, average = 'macro'))
print("micro f1 scoore :",metrics.f1_score(data_test_label, predictions, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(data_test_label,predictions))
print("Precision recall report :\n",metrics.classification_report(data_test_label, predictions))